In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('/home/thiri/Documents/RT Projects/Version1/dataset/april_plansummary.csv')

/tmp/ipykernel_118547/1897051147.py:1: DtypeWarning: Columns (2,13,18,20,24,26,27,40,46,47,49,50,67,76,83,86,88,91,94,95,96,97,98) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('/home/thiri/Documents/RT Projects/Version1/dataset/april_plansummary.csv')


In [3]:
data.shape

(953917, 102)

In [4]:
active_terminate=data[(data['Status']=='TERMINATED') | (data['Status']=='ACTIVE')]

In [5]:
active_terminate.shape

(823358, 102)

In [6]:
active=active_terminate[active_terminate['Status']=='ACTIVE']

In [7]:
terminate=active_terminate[active_terminate['Status']=='TERMINATED']

In [8]:
terminate.shape

(460016, 102)

In [9]:
terminate['State Reason'].value_counts()

State Reason
Terminate             243180
Terminate [Unpaid]    161605
Bad debt               53499
Restart                 1017
Name: count, dtype: int64

In [10]:
terminate['State Reason']=='Restart'

3579      False
3623      False
3757      False
4668      False
4676      False
          ...  
951902    False
952100    False
952208    False
952668    False
952854    False
Name: State Reason, Length: 460016, dtype: bool

### Filter terminated data with time

In [11]:
terminate['Billing Terminate Date'] = pd.to_datetime(terminate['Billing Terminate Date'], errors='coerce')

# Define the start and end dates
start_date = pd.to_datetime('2025-03-27')
end_date = pd.to_datetime('2025-04-30')

filtered_terminate = terminate[
    (terminate['Billing Terminate Date'] >= start_date) &
    (terminate['Billing Terminate Date'] <= end_date)
]

/tmp/ipykernel_39241/2448959413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  terminate['Billing Terminate Date'] = pd.to_datetime(terminate['Billing Terminate Date'], errors='coerce')


In [12]:
filtered_terminate.shape

(2767, 102)

In [13]:
filtered_terminate['State Reason'].value_counts()

State Reason
Terminate             1954
Terminate [Unpaid]     763
Restart                 50
Name: count, dtype: int64

In [14]:
filtered_terminate_2=filtered_terminate[~(filtered_terminate['State Reason']=='Restart')]

In [15]:
filtered_terminate_2.shape

(2717, 102)

In [16]:
filtered_terminate_2['Customer ID'].nunique()

1763

In [17]:
combine_active_terminate=pd.concat([active, filtered_terminate_2])

In [18]:
combine_active_terminate.shape

(366059, 102)

In [19]:
combine_active_terminate['Customer ID'].nunique()

222804

#### For each Customer ID, if all associated plan_summary records have a Status of only 'ACTIVE', the customer is considered 'ACTIVE'.

#### However, if there is at least one record with the Status 'TERMINATED', within ‘Billing Terminate Date’ between (March 27,2025) to 
#### (April,30,2025), the customer is classified as 'TERMINATED'.

In [20]:
def determine_status(status_series):
    if all(status == 'ACTIVE' for status in status_series):
        return 'ACTIVE'
    else:
        return 'TERMINATED'

summary_status = (
    combine_active_terminate
    .groupby('Customer ID')['Status']
    .agg(determine_status)
    .reset_index()
)


In [21]:
summary_status

Customer ID  Status
0         2000006.0  ACTIVE
1         2000026.0  ACTIVE
2         2000027.0  ACTIVE
3         2000042.0  ACTIVE
4         2000057.0  ACTIVE
...             ...     ...
222799    2589281.0  ACTIVE
222800    2589282.0  ACTIVE
222801    2589283.0  ACTIVE
222802    2589284.0  ACTIVE
222803    2589285.0  ACTIVE

[222804 rows x 2 columns]

In [22]:
summary_status['Status'].value_counts()

Status
ACTIVE        221041
TERMINATED      1763
Name: count, dtype: int64

In [23]:
summary_status.to_csv('temp_files/summary_status.csv')